# Repeated-measures ANOVA with two within-subject factors

Based on [Breska & Deouell (2017)](https://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.2001665).

This is an R notebook based on what I used to simulate the data and generate the figures for Experiment 2 in my lecture on multi-way ANOVA. To run the code, click on each block (a "cell") of code and click the "play" button near the top of the page. There is also a button at the top of the page to run all of the cells (the two right arrows, or "fast-forward icon", at the top).

*Written with the help of Microsoft Copilot and Claude Haiku.*

In [ ]:
# Load necessary libraries
library(tidyverse)
library(afex)
library(emmeans)

# Increase plot size
options(repr.plot.width=12, repr.plot.height=7)

### Generate the data

In [ ]:
# Set seed for reproducibility
#set.seed(42)

# Number of subjects and levels (conditions)
n_subjects <- 21
targets <- c('Valid', 'Invalid')
n_targets <- length(targets)
temporal <- c('Rhythmic','Random')
n_temporal <- length(temporal)

# Mean values and standard deviation for each level
means <- array(c(225, 295, 280, 300), dim=c(2,2))
std_dev_sbj <- 115 # standard deviation across participants
std_dev_wthn <- 40 # standard deviation within participants

# Initialize empty data frame to store results
df <- data.frame(
  Subject = numeric(),
  Temporal = character(),
  Target = character(),
  Response_time = numeric()
)

# Simulate data
for (subject in 1:n_subjects) {
  sbj_avg <- rnorm(1, mean = 0, sd = std_dev_sbj)
  for (trg in 1:n_targets) {
      for (tmp in 1:n_temporal) {
        value <- rnorm(1, mean = means[trg,tmp], sd = std_dev_wthn)
        value <- value + sbj_avg

        # Add row to data frame
        df <- rbind(df, data.frame(
          Subject = subject,
          Temporal = temporal[tmp],
          Target = targets[trg],
          Response_time = value
        ))
      }
  }
}

glimpse(df)

In [ ]:
# Plot the data
ggplot(df, aes(x = Target, y = Response_time, group = Subject)) +
  geom_line(aes(color = Subject)) +
  geom_point() +
  facet_wrap(~ Temporal)
  labs(x = "Target", y = "Response time (ms)") +
  theme_grey(base_size=20)

In [ ]:
# Use tukeyboxplot
ggplot(data=df, aes(x = Temporal, y = Response_time, fill = Target)) +
    geom_boxplot() +
    geom_jitter(position=position_jitterdodge(jitter.width=0.1, dodge.width=0.8)) +
    theme_grey(base_size=20)

### Run the repeated-measures ANOVA

In [ ]:
# Run the repeated-measures ANOVA
anova_results <- aov_car(Response_time ~ Error(Subject/(Temporal * Target)), data=df)
summary_aov <- summary(anova_results)
summary_aov

In [ ]:
anova_results

In [ ]:
# Calculate the between-subjects sum-of-squares
SS_total <- sum((df$Response_time - mean(df$Response_time))^2)
    # This will be different than what you see in the ANOVA table above. 
    # R displays the sum-of-squares without subtracting the mean. 
    # Calculating the total SS centered on the mean is more standard.
sbj_mean <- aggregate(Response_time ~ Subject, df, mean)
SS_sbj <- sum((sbj_mean$Response_time - mean(df$Response_time))^2) * n_targets * n_temporal
SS_within <- SS_total - SS_sbj

cat(sprintf("Total SS = %.2f\n", SS_total))
cat(sprintf("Between-subject SS = %.2f\n", SS_sbj))
cat(sprintf("Within-subject SS = %.2f\n", SS_within))

### Multiple comparisons

In [ ]:
# Multiple comparisons test with emmeans
emmeans_results <- emmeans(anova_results, specs = pairwise ~ Target|Temporal, adj='bonf')
print(emmeans_results$contrasts)